# LangChain + MongoDB Text-to-MQL Demo

In [ ]:
!pip install langchain langchain-mongodb pymongo python-dotenv openai

In [ ]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"))
db = client["customer_support"]
tickets = db["tickets"]
tickets.insert_many([
    {"ticket_id": 1, "customer_name": "Alice", "issue": "Refund request", "status": "unresolved", "date": "2025-06-25"},
    {"ticket_id": 2, "customer_name": "Bob", "issue": "Tech support", "status": "resolved", "date": "2025-06-20"},
    {"ticket_id": 3, "customer_name": "Charlie", "issue": "Billing issue", "status": "unresolved", "date": "2025-06-28"}
]) if tickets.estimated_document_count() == 0 else None


In [ ]:
from langchain_mongodb import MongoDBAgentToolkit
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate

toolkit = MongoDBAgentToolkit(
    client=client,
    database_name="customer_support",
    llm_model="openai",
    api_key=os.getenv("OPENAI_API_KEY")
)

prompt = PromptTemplate.from_template("""
You are a support assistant. Use the tools to answer this:
Query: {query}
Tools: {tools}
""")

agent = create_react_agent(toolkit.llm, toolkit.get_tools(), prompt)
executor = AgentExecutor(agent=agent, tools=toolkit.get_tools(), verbose=True)

def ask(q): return executor.run(query=q)
ask("Show me unresolved tickets from the last week")
